In [1]:
import numpy as np
import tensorflow as tf
from feedforward import FeedForward
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

In [2]:
digits = datasets.load_digits()

In [3]:
data = digits.data
targets = digits.target

In [4]:
data.shape

(1797, 64)

In [5]:
targets.shape

(1797,)

In [6]:
set(targets)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [7]:
targets = targets.reshape(-1,1)
data = StandardScaler().fit_transform(data)
idx = np.arange(data.shape[0])
np.random.shuffle(idx)
train_data = data[idx[:1000]]
train_targets = targets[idx[:1000]]
train_targets = LabelBinarizer().fit_transform(train_targets)
validation_data = data[idx[1000:1350]]
validation_targets = targets[idx[1000:1350]]
test_data = data[idx[1350:]]
test_targets = targets[idx[1350:]]

In [8]:
tf.reset_default_graph()
mynnet = FeedForward([data.shape[1], 100, 80, train_targets.shape[1]],
                     activation=tf.nn.relu)
outputs = mynnet.get_output()
labels = mynnet.get_targets()
cost = tf.nn.softmax_cross_entropy_with_logits(outputs, labels)
cost = tf.reduce_mean(cost)
opt = tf.train.AdamOptimizer(learning_rate=0.001)
opt = opt.minimize(cost)

In [9]:
with tf.Session() as sess:
    tf.initialize_all_variables().run(session=sess)
    batch_size = 50
    num_epochs = 100
    num_iter = train_data.shape[0] / batch_size
    train_idx = np.arange(train_data.shape[0])
    train_keep_probs = [0.8, 0.5, 0.5]
    for epoch in xrange(1, num_epochs + 1):
        for i in xrange(num_iter):
            batch_idx = train_idx[i*batch_size: (i*batch_size)+batch_size]
            batch_data = train_data[batch_idx]
            batch_targets = train_targets[batch_idx]

            feed_dict = mynnet.get_train_feed_dict(inputs=batch_data,
                                             targets=batch_targets,
                                             keep_probs=train_keep_probs)
            xentropy, _ = sess.run([cost, opt], feed_dict = feed_dict)
            
            if (i+1) % 5 == 0:
                print 'Epoch: %d | iter: %d | loss: %f'%(epoch, i+1, xentropy)
        print '\n       ----- Validation report after epoch %d -------'%epoch
        feed_dict = mynnet.get_test_feed_dict(inputs=validation_data)
        preds = sess.run([outputs], feed_dict=feed_dict)[0]
        preds = tf.nn.softmax(preds).eval(session=sess)
        preds = np.argmax(preds, 1)
        print classification_report(validation_targets, preds)
    
    print '         ------- Test report -------'
    feed_dict = mynnet.get_test_feed_dict(inputs=test_data)
    preds = sess.run([outputs], feed_dict=feed_dict)[0]
    preds = tf.nn.softmax(preds).eval(session=sess)
    preds = np.argmax(preds, 1)
    print classification_report(test_targets, preds)

    print '          ------- Full Data report -------'
    feed_dict = mynnet.get_test_feed_dict(inputs=data)
    preds = sess.run([outputs], feed_dict=feed_dict)[0]
    preds = tf.nn.softmax(preds).eval(session=sess)
    preds = np.argmax(preds, 1)
    print classification_report(targets.reshape(-1, 1), preds)  

Epoch: 1 | iter: 5 | loss: 2.363441
Epoch: 1 | iter: 10 | loss: 2.098962
Epoch: 1 | iter: 15 | loss: 2.050537
Epoch: 1 | iter: 20 | loss: 1.842091

       ----- Validation report after epoch 1 -------
             precision    recall  f1-score   support

          0       0.90      0.80      0.85        45
          1       0.63      0.50      0.56        24
          2       0.59      0.90      0.71        30
          3       0.44      0.85      0.58        41
          4       1.00      0.28      0.43        36
          5       0.00      0.00      0.00        40
          6       0.86      0.95      0.90        39
          7       0.32      0.97      0.48        31
          8       0.67      0.21      0.32        29
          9       0.60      0.17      0.27        35

avg / total       0.60      0.57      0.52       350

Epoch: 2 | iter: 5 | loss: 1.688156
Epoch: 2 | iter: 10 | loss: 1.661913
Epoch: 2 | iter: 15 | loss: 1.433147
Epoch: 2 | iter: 20 | loss: 1.414702

       -----